In [ ]:
from binance.client import Client
import time
import gate_api
from gate_api.exceptions import ApiException, GateApiException


# API anahtarlarınızı kullanarak bir Binance client nesnesi oluşturun
client = Client(api_key='key', api_secret='secret')



# Toplu işlemleri işleyerek toplam satış ve alış miktarlarını hesapla

while True:
    
    # PEPE/USDT için son 5 toplu işlemi sorgula
    aggregate_trades = client.get_aggregate_trades(symbol='BTCUSDT', limit=100)
    current_price = client.get_symbol_ticker(symbol="BTCUSDT")
    total_sold = 0
    total_bought = 0

    for trade in aggregate_trades:
        amount = float(trade['p']) * float(trade['q'])  # Fiyat x Miktar
        if trade['m']:  # Eğer 'm' anahtarı True ise, işlem satıcı tarafından başlatıldı ve bu bir satış işlemidir
            total_sold += amount
        else:  # Eğer 'm' anahtarı False ise, işlem alıcı tarafından başlatıldı ve bu bir alım işlemidir
            total_bought += amount

    # Toplam satış ve alış miktarlarını yazdır
    Denge = total_bought - total_sold
    print(f"Satış {total_sold:.1f}  Alış: {total_bought:.1f}, Fiyat: {current_price['price']}, Denge: {Denge: .1f}")
    #print(f"Toplamda {total_bought:.1f} USDT değerinde PEPE satın alındı.")
    
    time.sleep(6)

Satış 145373.8  Alış: 17582.9, Fiyat: 68040.02000000, Denge: -127790.9


## YOU CAN DO THIS!!

In [3]:
from binance.client import Client
import time
import gate_api
import requests
from gate_api.exceptions import ApiException, GateApiException
import numpy as np

# API anahtarları
BINANCE_API_KEY = 'key'
BINANCE_API_SECRET = 'secret'
GATEIO_API_KEY = 'key'
GATEIO_API_SECRET = 'secret'

# Sabitler
SYMBOL = 'WIF_USDT'
BINANCE_SYMBOL = 'WIFUSDT'
SLEEP_DURATION = 3 # saniye

# Binance ve Gate.io için API client nesnelerini oluşturma
client = Client(api_key=BINANCE_API_KEY, api_secret=BINANCE_API_SECRET)
configuration = gate_api.Configuration(key=GATEIO_API_KEY, secret=GATEIO_API_SECRET)
gateio_api_client = gate_api.ApiClient(configuration)
gateio_spot_api = gate_api.SpotApi(gateio_api_client)

def calculate_ema(closes, period=1):
    ema = np.zeros_like(closes)
    ema[0] = closes[0]  # Başlangıç değeri, ilk kapanış fiyatıyla aynı
    # EMA hesaplaması
    alpha = 2 / (period + 1)
    for i in range(1, len(closes)):
        ema[i] = closes[i] * alpha + ema[i - 1] * (1 - alpha)
    return ema[-1]

def calculate_ema2(closes, period=2):
    ema = np.zeros_like(closes)
    ema[0] = closes[0]  # Başlangıç değeri, ilk kapanış fiyatıyla aynı
    # EMA hesaplaması
    alpha = 2 / (period + 1)
    for i in range(1, len(closes)):
        ema[i] = closes[i] * alpha + ema[i - 1] * (1 - alpha)
    return ema[-1]

def calculate_rsi(closes, period=2):
    deltas = np.diff(closes)
    seed = deltas[:period]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down
    rsi = np.zeros_like(closes)
    rsi[:period] = 100. - 100. / (1. + rs)
    for i in range(period, len(closes)):
        delta = deltas[i - 1]  # `i` öğesiyle bir önceki kapanış arasındaki fark
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period
        rs = up / down
        rsi[i] = 100. - 100. / (1. + rs)
    return rsi[-1]

def get_balance_difference_gateio(api_client, spot_api, symbol):
    current_price = float(spot_api.list_tickers(currency_pair=symbol)[0].last)
    return current_price

def get_balance_difference_binance(client, symbol):
    current_price = client.get_symbol_ticker(symbol=symbol)
    return current_price['price']

SIMULATION_BALANCE_USDT = 10.0  # Başlangıç simülasyon bakiyesi (USDT)
SIMULATION_BTC = 0  # Başlangıçta BTC miktarı

try:
    while True:
        #gateio ve binance de priceları ve balanceyi hesapla!
        price_gateio= get_balance_difference_gateio(gateio_api_client, gateio_spot_api, SYMBOL)
        price_binance = get_balance_difference_binance(client, BINANCE_SYMBOL)
        
        #pricelar str geliyor floata çevirdik.
        price_gateio_f = float(price_gateio)
        price_binance_f = float(price_binance)
        
        #Borsalar arası fiyat farkından alım satım emri vermeye çalıştık, alım satım farkları her iki borsa içinde toplayıp denge_toplami diye bir değişken türettik
        fiyat_farki = price_binance_f - price_gateio_f
        #denge_toplami = balance_diff_gateio_f + balance_diff_binance_f

        # 1 dakikalık RSI hesaplama
        klines = client.get_historical_klines(BINANCE_SYMBOL, '1m', '100 minutes ago UTC')
        closes = np.array([float(kline[4]) for kline in klines])
        rsi = calculate_rsi(closes, period=2)
        
        # mum en yüksek kapanış fiyatları çekme
        latest_candle = klines[-1]
        high = float(latest_candle[2])  # En yüksek fiyat
        low = float(latest_candle[3])   # En düşük fiyat
        open_price = float(latest_candle[1])  # Açılış fiyatı
        close = float(latest_candle[4])  # Kapanış fiyatı
        #print(f"En Yüksek Fiyat: {high}, En Düşük Fiyat: {low}, Açılış Fiyatı: {open_price}, Kapanış Fiyatı: {close}")
        
        #EMA Hesaplama
        ema = calculate_ema(closes, period=1)
        ema2 = calculate_ema2(closes, period=2)

        # Tüm verilerin birleştirilerek yazdırılması
        print(f"Fiyat:{price_gateio_f:.4f}, Fark: {fiyat_farki:.4f}, RSI:{rsi:.4f}, EMA1: {ema:.4f}, EMA2: {ema2:.4f}")
        #Alım Sinyal
        if ema >= ema2 and SIMULATION_BALANCE_USDT >= 5 and rsi < 30:  
            bought_btc = 10 / price_gateio_f
            SIMULATION_BALANCE_USDT -= 10
            SIMULATION_BTC += bought_btc
            print(f"Alım yapıldı: {bought_btc} BTC, Yeni bakiye: {SIMULATION_BALANCE_USDT} USDT, BTC: {SIMULATION_BTC}")
        
        # Satım sinyali
        elif ema2 > ema and SIMULATION_BTC * price_gateio_f >= 10 and rsi > 70:
            sold_usdt = SIMULATION_BTC * price_gateio_f
            fee = sold_usdt * 0.002
            net_usdt = sold_usdt - fee
            SIMULATION_BALANCE_USDT += net_usdt
            SIMULATION_BTC = 0
            print(f"Satım yapıldı: {sold_usdt} USDT, Yeni bakiye: {SIMULATION_BALANCE_USDT} USDT, BTC: {SIMULATION_BTC}")
        
        time.sleep(SLEEP_DURATION)
        
except KeyboardInterrupt:
    print("Döngü manuel olarak durduruldu.")
except Exception as e:
    print(f"Bir hata oluştu: {e}")


Fiyat:3.6135, Fark: 0.0005, RSI:31.0774, EMA1: 3.6140, EMA2: 3.6144
Fiyat:3.6135, Fark: 0.0016, RSI:51.5101, EMA1: 3.6176, EMA2: 3.6168
Döngü manuel olarak durduruldu.
